# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [1]:
# import libraries
import pandas as pd

In [2]:
# load data from database
from sqlalchemy import create_engine

engine = create_engine('sqlite:///data/DisasterResponse_processed.db')
connection = engine.connect()

tblName = engine.table_names()[0]
print(tblName)

InsertTableName


In [3]:
df = pd.read_sql_table(tblName, connection)

X = df['message']
Y = df.loc[:, df.columns != 'message']

In [4]:
Y.head(5)

,id,original,genre,related,request,offer,aid_related,medical_help,medical_products,search_and_rescue,...,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report
0,2,Un front froid se retrouve sur Cuba ce matin. ...,direct,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,7,Cyclone nan fini osinon li pa fini,direct,1,0,0,1,0,0,0,...,0,0,1,0,1,0,0,0,0,0
2,8,"Patnm, di Maryani relem pou li banm nouvel li ...",direct,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,9,UN reports Leogane 80-90 destroyed. Only Hospi...,direct,1,1,0,1,0,1,0,...,0,0,0,0,0,0,0,0,0,0
4,12,facade ouest d Haiti et le reste du pays aujou...,direct,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [5]:
### index, id does not seem to have any factual implication 
### original seems to be replica of message but in other languages
### but what about genre? 
Y.genre.value_counts()

news      13036
direct    10634
social     2358
Name: genre, dtype: int64

In [6]:
Y.sum(axis=0)

id                                                                396933688
genre                     directdirectdirectdirectdirectdirectdirectdire...
related                                                               19906
request                                                                4474
offer                                                                   118
aid_related                                                           10860
medical_help                                                           2084
medical_products                                                       1313
search_and_rescue                                                       724
security                                                                471
military                                                                860
child_alone                                                               0
water                                                                  1672
food        

#### Dropping non useful columns 

<p>Genre - The value indicates the source where does the messaege derives from
It may seem to matter how importance/priority we treat the messages 
but if we treat each message with equal importance, then this message
may not seem important(the criteria for project seems to be classify responses to particular relief organization for handle</p>

<p> Child Alone - there is no cases which indicate child alone. 100% of test case will indicate no help in this category is needed </p>

<p> id - seem to be the cases no and provided no useful information </p>

<p> original - seem to be original help message which also provide non useful information </p>

In [7]:
### drop some columns that does not seem useful
# Y = Y.drop(columns=['index', 'id', 'original', 'genre'], axis=1)
Y = Y.drop(columns=[ 'id', 'original', 'genre', 'child_alone'], axis=1)

### 2. Write a tokenization function to process your text data

In [8]:
from nltk.tokenize import word_tokenize
from nltk.tokenize import sent_tokenize

import nltk
import re
from nltk.corpus import stopwords
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.tokenize import word_tokenize

nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')

stop_words = stopwords.words("english")
lemmatizer = WordNetLemmatizer()

def tokenize(text):
    # normalize case and remove punctuation
    text = re.sub(r"[^a-zA-Z0-9]", " ", text.lower())
    
    # tokenize text
    tokens = word_tokenize(text)
    
    # lemmatize andremove stop words
    tokens = [lemmatizer.lemmatize(word) for word in tokens if word not in stop_words]

    return tokens

text = "Dr. Smith graduated from the University of Washington. He later started an analytics firm called Lux, which catered to enterprise customers."
print(tokenize(text))

[nltk_data] Downloading package stopwords to /home/monki/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /home/monki/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /home/monki/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


['dr', 'smith', 'graduated', 'university', 'washington', 'later', 'started', 'analytics', 'firm', 'called', 'lux', 'catered', 'enterprise', 'customer']


### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [9]:
from sklearn.multioutput import MultiOutputClassifier
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import LinearSVC
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier, GradientBoostingClassifier

#### To build a classifier to separate them into class
#### We try to choose from ensemble type of learning classifier, trees

pipeline = Pipeline([
        ('feat', FeatureUnion([
            ('pipeline', Pipeline([
                ('vect', CountVectorizer(tokenizer=tokenize)),
                ('tfidf', TfidfTransformer())
                ])
            )            
            ])
        ),
        ('clf', MultiOutputClassifier(AdaBoostClassifier()))
])


### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [10]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.3, random_state=42, shuffle=True)

# model = build_model()
pipeline.fit(X_train, y_train)

Pipeline(steps=[('feat',
                 FeatureUnion(transformer_list=[('pipeline',
                                                 Pipeline(steps=[('vect',
                                                                  CountVectorizer(tokenizer=<function tokenize at 0x7f10413483a0>)),
                                                                 ('tfidf',
                                                                  TfidfTransformer())]))])),
                ('clf', MultiOutputClassifier(estimator=AdaBoostClassifier()))])

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [11]:
from sklearn.metrics import classification_report

y_pred = pipeline.predict(X_test)

for category in y_test:
    print('Category: {}'.format(category))
    idx = y_test.columns.get_loc(category)
    print(classification_report(y_test[category], y_pred[:,idx]))

Category: related
              precision    recall  f1-score   support

           0       0.70      0.17      0.27      1868
           1       0.79      0.98      0.87      5941

    accuracy                           0.78      7809
   macro avg       0.75      0.57      0.57      7809
weighted avg       0.77      0.78      0.73      7809

Category: request
              precision    recall  f1-score   support

           0       0.90      0.96      0.93      6476
           1       0.75      0.50      0.60      1333

    accuracy                           0.89      7809
   macro avg       0.83      0.73      0.77      7809
weighted avg       0.88      0.89      0.88      7809

Category: offer
              precision    recall  f1-score   support

           0       1.00      1.00      1.00      7775
           1       0.00      0.00      0.00        34

    accuracy                           0.99      7809
   macro avg       0.50      0.50      0.50      7809
weighted avg       0.9

              precision    recall  f1-score   support

           0       0.99      1.00      0.99      7721
           1       0.38      0.06      0.10        88

    accuracy                           0.99      7809
   macro avg       0.69      0.53      0.55      7809
weighted avg       0.98      0.99      0.98      7809

Category: earthquake
              precision    recall  f1-score   support

           0       0.98      0.99      0.98      7075
           1       0.88      0.80      0.84       734

    accuracy                           0.97      7809
   macro avg       0.93      0.89      0.91      7809
weighted avg       0.97      0.97      0.97      7809

Category: cold
              precision    recall  f1-score   support

           0       0.99      1.00      0.99      7640
           1       0.74      0.42      0.54       169

    accuracy                           0.98      7809
   macro avg       0.86      0.71      0.76      7809
weighted avg       0.98      0.98     

### 6. Improve your model
Use grid search to find better parameters. 

In [12]:
### get parameters we can tune the pipeline for
pipeline.get_params()

{'memory': None,
 'steps': [('feat', FeatureUnion(transformer_list=[('pipeline',
                                   Pipeline(steps=[('vect',
                                                    CountVectorizer(tokenizer=<function tokenize at 0x7f10413483a0>)),
                                                   ('tfidf',
                                                    TfidfTransformer())]))])),
  ('clf', MultiOutputClassifier(estimator=AdaBoostClassifier()))],
 'verbose': False,
 'feat': FeatureUnion(transformer_list=[('pipeline',
                                 Pipeline(steps=[('vect',
                                                  CountVectorizer(tokenizer=<function tokenize at 0x7f10413483a0>)),
                                                 ('tfidf',
                                                  TfidfTransformer())]))]),
 'clf': MultiOutputClassifier(estimator=AdaBoostClassifier()),
 'feat__n_jobs': None,
 'feat__transformer_list': [('pipeline',
   Pipeline(steps=[('vec



Reference:
<ol>
    
<li>https://towardsdatascience.com/journey-to-the-center-of-multi-label-classification-384c40229bff</li>

<li>https://medium.datadriveninvestor.com/a-survey-of-evaluation-metrics-for-multilabel-classification-bb16e8cd41cd</li>
</ol>

A very important consideration to make for GridSearch is to consider the scoring it should use

In [13]:
from sklearn.model_selection import GridSearchCV

parameters = {
    'clf__estimator__learning_rate': [0.01, 0.02, 0.05],
    'clf__estimator__n_estimators': [10, 20, 40]
}

cv =  GridSearchCV(pipeline, param_grid=parameters)

cv.fit(X_train,y_train)

GridSearchCV(estimator=Pipeline(steps=[('feat',
                                        FeatureUnion(transformer_list=[('pipeline',
                                                                        Pipeline(steps=[('vect',
                                                                                         CountVectorizer(tokenizer=<function tokenize at 0x7f10413483a0>)),
                                                                                        ('tfidf',
                                                                                         TfidfTransformer())]))])),
                                       ('clf',
                                        MultiOutputClassifier(estimator=AdaBoostClassifier()))]),
             param_grid={'clf__estimator__learning_rate': [0.01, 0.02, 0.05],
                         'clf__estimator__n_estimators': [10, 20, 40]})

Find the average score of each model

In [14]:
cv.cv_results_

{'mean_fit_time': array([10.01680474, 16.01431508, 29.48751779,  9.52274823, 16.06361027,
        28.95693669,  9.52591019, 16.76436663, 30.99021544]),
 'std_fit_time': array([0.45449525, 0.25372159, 0.30646214, 0.25938091, 0.24613484,
        0.19806731, 0.21247432, 0.32898429, 0.98404879]),
 'mean_score_time': array([0.95984011, 1.04767079, 1.46789088, 0.91486173, 1.09262924,
        1.48913202, 0.87520709, 1.14027653, 1.51674128]),
 'std_score_time': array([0.07910343, 0.01612497, 0.02369736, 0.07814659, 0.06742354,
        0.11542645, 0.00744056, 0.03737368, 0.10174188]),
 'param_clf__estimator__learning_rate': masked_array(data=[0.01, 0.01, 0.01, 0.02, 0.02, 0.02, 0.05, 0.05, 0.05],
              mask=[False, False, False, False, False, False, False, False,
                    False],
        fill_value='?',
             dtype=object),
 'param_clf__estimator__n_estimators': masked_array(data=[10, 20, 40, 10, 20, 40, 10, 20, 40],
              mask=[False, False, False, False, Fals

Finding the best parameters

In [15]:
print(cv.best_params_)

{'clf__estimator__learning_rate': 0.05, 'clf__estimator__n_estimators': 40}


In [16]:
print (cv.best_estimator_)

optimised_model = cv.best_estimator_


Pipeline(steps=[('feat',
                 FeatureUnion(transformer_list=[('pipeline',
                                                 Pipeline(steps=[('vect',
                                                                  CountVectorizer(tokenizer=<function tokenize at 0x7f10413483a0>)),
                                                                 ('tfidf',
                                                                  TfidfTransformer())]))])),
                ('clf',
                 MultiOutputClassifier(estimator=AdaBoostClassifier(learning_rate=0.05,
                                                                    n_estimators=40)))])


### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [17]:
y_pred = optimised_model.predict(X_test)

for category in y_test:
    print('Category: {}'.format(category))
    idx = y_test.columns.get_loc(category)
    print(classification_report(y_test[category], y_pred[:,idx]))


Category: related
              precision    recall  f1-score   support

           0       0.00      0.00      0.00      1868
           1       0.76      1.00      0.86      5941

    accuracy                           0.76      7809
   macro avg       0.38      0.50      0.43      7809
weighted avg       0.58      0.76      0.66      7809

Category: request
              precision    recall  f1-score   support

           0       0.85      0.99      0.92      6476
           1       0.83      0.15      0.26      1333

    accuracy                           0.85      7809
   macro avg       0.84      0.57      0.59      7809
weighted avg       0.85      0.85      0.80      7809

Category: offer
              precision    recall  f1-score   support

           0       1.00      1.00      1.00      7775
           1       0.00      0.00      0.00        34

    accuracy                           1.00      7809
   macro avg       0.50      0.50      0.50      7809
weighted avg       0.9

/home/monki/anaconda3/envs/data_scientist/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1221: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


              precision    recall  f1-score   support

           0       0.99      1.00      0.99      7721
           1       1.00      0.01      0.02        88

    accuracy                           0.99      7809
   macro avg       0.99      0.51      0.51      7809
weighted avg       0.99      0.99      0.98      7809

Category: earthquake
              precision    recall  f1-score   support

           0       0.97      0.99      0.98      7075
           1       0.89      0.66      0.76       734

    accuracy                           0.96      7809
   macro avg       0.93      0.82      0.87      7809
weighted avg       0.96      0.96      0.96      7809

Category: cold
              precision    recall  f1-score   support

           0       0.98      1.00      0.99      7640
           1       0.69      0.05      0.10       169

    accuracy                           0.98      7809
   macro avg       0.84      0.53      0.54      7809
weighted avg       0.97      0.98     

### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

### Try With Another Classifier
#### 1) We try wth another ensemble based classifier RandomForestClassifier

In [18]:
pipeline2 = Pipeline([
        ('feat', FeatureUnion([
            ('pipeline', Pipeline([
                ('vect', CountVectorizer(tokenizer=tokenize)),
                ('tfidf', TfidfTransformer())
            ]))            
        ])),
        ('clf', MultiOutputClassifier(RandomForestClassifier()))
])


In [19]:
pipeline2.fit(X_train,y_train)

y_pred = pipeline2.predict(X_test)

for category in y_test:
    print('Category: {}'.format(category))
    idx = y_test.columns.get_loc(category)
    print(classification_report(y_test[category], y_pred[:,idx]))

Category: related
              precision    recall  f1-score   support

           0       0.72      0.39      0.51      1868
           1       0.83      0.95      0.89      5941

    accuracy                           0.82      7809
   macro avg       0.78      0.67      0.70      7809
weighted avg       0.81      0.82      0.80      7809

Category: request
              precision    recall  f1-score   support

           0       0.90      0.98      0.94      6476
           1       0.81      0.50      0.62      1333

    accuracy                           0.89      7809
   macro avg       0.86      0.74      0.78      7809
weighted avg       0.89      0.89      0.88      7809

Category: offer
              precision    recall  f1-score   support

           0       1.00      1.00      1.00      7775
           1       0.00      0.00      0.00        34

    accuracy                           1.00      7809
   macro avg       0.50      0.50      0.50      7809
weighted avg       0.9

/home/monki/anaconda3/envs/data_scientist/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1221: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


              precision    recall  f1-score   support

           0       0.99      1.00      0.99      7721
           1       0.67      0.02      0.04        88

    accuracy                           0.99      7809
   macro avg       0.83      0.51      0.52      7809
weighted avg       0.99      0.99      0.98      7809

Category: earthquake
              precision    recall  f1-score   support

           0       0.98      0.99      0.98      7075
           1       0.89      0.78      0.83       734

    accuracy                           0.97      7809
   macro avg       0.93      0.88      0.91      7809
weighted avg       0.97      0.97      0.97      7809

Category: cold
              precision    recall  f1-score   support

           0       0.98      1.00      0.99      7640
           1       0.89      0.09      0.17       169

    accuracy                           0.98      7809
   macro avg       0.93      0.55      0.58      7809
weighted avg       0.98      0.98     

#### 2) We try wth Linear based classifier LinearSVC

In [20]:
pipeline3 = Pipeline([
        ('feat', FeatureUnion([
            ('pipeline', Pipeline([
                ('vect', CountVectorizer(tokenizer=tokenize)),
                ('tfidf', TfidfTransformer())
            ]))            
        ])),
          ('clf', MultiOutputClassifier(LinearSVC()))
])


In [21]:
y_train

,related,request,offer,aid_related,medical_help,medical_products,search_and_rescue,security,military,water,...,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report
20601,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
6775,1,1,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
24412,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
20086,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9397,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21575,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5390,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
860,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
15795,1,0,0,0,0,0,0,0,0,0,...,0,0,1,1,0,0,0,0,0,0


In [22]:
pipeline3.fit(X_train,y_train)

y_pred2 = pipeline3.predict(X_test)

for category in y_test:
    print('Category: {}'.format(category))
    idx = y_test.columns.get_loc(category)
    print(classification_report(y_test[category], y_pred2[:,idx]))

Category: related
              precision    recall  f1-score   support

           0       0.66      0.52      0.58      1868
           1       0.86      0.92      0.89      5941

    accuracy                           0.82      7809
   macro avg       0.76      0.72      0.73      7809
weighted avg       0.81      0.82      0.81      7809

Category: request
              precision    recall  f1-score   support

           0       0.92      0.96      0.94      6476
           1       0.74      0.59      0.66      1333

    accuracy                           0.89      7809
   macro avg       0.83      0.77      0.80      7809
weighted avg       0.89      0.89      0.89      7809

Category: offer
              precision    recall  f1-score   support

           0       1.00      1.00      1.00      7775
           1       0.00      0.00      0.00        34

    accuracy                           1.00      7809
   macro avg       0.50      0.50      0.50      7809
weighted avg       0.9

/home/monki/anaconda3/envs/data_scientist/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1221: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


              precision    recall  f1-score   support

           0       0.99      1.00      1.00      7721
           1       0.66      0.26      0.37        88

    accuracy                           0.99      7809
   macro avg       0.82      0.63      0.68      7809
weighted avg       0.99      0.99      0.99      7809

Category: earthquake
              precision    recall  f1-score   support

           0       0.97      0.99      0.98      7075
           1       0.88      0.73      0.80       734

    accuracy                           0.97      7809
   macro avg       0.93      0.86      0.89      7809
weighted avg       0.96      0.97      0.96      7809

Category: cold
              precision    recall  f1-score   support

           0       0.99      1.00      0.99      7640
           1       0.70      0.34      0.46       169

    accuracy                           0.98      7809
   macro avg       0.84      0.67      0.72      7809
weighted avg       0.98      0.98     

#### If we use weighted average as the deciding factor, we find that RandomForestClassifer or pipeline2 performs better

### 9. Export your model as a pickle file

In [26]:
import pickle

filename = "classifier.pkl" 

with open(filename, 'wb') as file:  
    pickle.dump(pipeline2,file)

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.